In [1]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

## Model

In [2]:
import torch
from transformers import pipeline

model_name = "openai/whisper-medium"

model = pipeline(
    "automatic-speech-recognition", 
    model=model_name,
    torch_dtype=torch.float16, 
    device="mps"
)

Device set to use mps


## Files

In [3]:
from tqdm.notebook import tqdm

import importlib
import pandas as pd
import get_files as gf

In [27]:
importlib.reload(gf)

<module 'get_files' from '/Users/salgu/Workspace/1_project/cow/get_files.py'>

In [5]:
directory_path = "./audio"
extensions_to_find = ['.mp3']

grouped_files = gf.get_files_grouped_by_first_subdirectory(
    directory_path, 
    extensions_to_find
)

## Process

In [20]:
sample = {
    'text': ' 싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정확한 경기 결과는 경광판을 통해서 다시 한번 확인해 주시기 바랍니다 확정 전까지는 우건을 구기거나 부리지 마시기 바랍니다.',
    'chunks': [
        {'timestamp': (0.0, 5.84), 'text': ' 싸울 준비를 하고 있죠'},
        {'timestamp': (7.08, 9.68), 'text': ' 자 양전'},
        {'timestamp': (18.72, 25.36), 'text': ' 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정확한 경기 결과는'},
        {'timestamp': (25.36, 29.8), 'text': ' 경광판을 통해서 다시 한번 확인해 주시기 바랍니다'},
        {'timestamp': (0.0, 4.2), 'text': ' 확정 전까지는 우건을 구기거나 부리지 마시기 바랍니다.'}
    ]
}

In [28]:
for first_subdir, files in tqdm(grouped_files.items()):
    print(f"Working Directory: {first_subdir}")
    transcriptions = []
    for file in tqdm(files):
        # transcription = model(
        #     file,
        #     return_timestamps=True
        # )
        # transcriptions.append(transcription)
        transcriptions.append(
            {
                "file": file,
                # "transcription": "Sample transcription for testing"
                "data": sample
            }
        )
    df = gf.create_dataframe_from_list(transcriptions)
    # df.to_csv(f"transcriptions_{first_subdir}.csv", index=False)
    df.to_csv(f"transcriptions_test_{first_subdir}.csv", index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

Working Directory: 2022


  0%|          | 0/2 [00:00<?, ?it/s]

Working Directory: 2023


  0%|          | 0/2 [00:00<?, ?it/s]

Working Directory: 2024


  0%|          | 0/4 [00:00<?, ?it/s]

## Result

In [29]:
sample_df = pd.read_csv("transcriptions_test_2024.csv")
sample_df.head()

,file,text,timestamp
0,./audio/2024/2024-01/2024.02.21_02_test.mp3,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."
1,./audio/2024/2024-01/2024.02.21_01_test.mp3,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."
2,./audio/2024/2024-02/2024.02.21_02_test.mp3,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."
3,./audio/2024/2024-02/2024.02.21_01_test.mp3,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."


In [30]:
sample_dict = sample_df.to_dict(orient="records")

In [31]:
sample_dict[0]

{'file': './audio/2024/2024-01/2024.02.21_02_test.mp3',
 'text': ' 싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정확한 경기 결과는 경광판을 통해서 다시 한번 확인해 주시기 바랍니다 확정 전까지는 우건을 구기거나 부리지 마시기 바랍니다.',
 'timestamp': '0.0 - 5.84:  싸울 준비를 하고 있죠, 7.08 - 9.68:  자 양전, 18.72 - 25.36:  제2경기 이렇게 마무리가 됐습니다. 이 경기의 정확한 경기 결과는, 25.36 - 29.8:  경광판을 통해서 다시 한번 확인해 주시기 바랍니다, 0.0 - 4.2:  확정 전까지는 우건을 구기거나 부리지 마시기 바랍니다.'}